In [ ]:
from collections import Counter
from tqdm import tqdm
import numpy as np
import collections
import nbimporter
import warnings

warnings.filterwarnings("ignore")
import Utilities as utility

In [ ]:
doc_summary_pair = utility.loadObj('test_doc_summary_pair')

In [ ]:
combined = []

In [ ]:
combined = []
for pair in tqdm(doc_summary_pair):
    try:
        doc = pair[0]
        sentences = doc.split('.')

        clean_text = []
        included_sentences = []
        for sentence in sentences:
            if (len(utility.simpleRemoveStopWords(sentence)) > 3):
                clean_text.append(utility.simpleRemoveStopWords(sentence))
                included_sentences.append(sentence)

        vocab = sum(clean_text,[])


        occurenceMatrix = utility.findOccurenceMatrix(clean_text, vocab)
        degreeWord = occurenceMatrix.sum(axis=1)
        freqWord = np.amax(occurenceMatrix, axis=1)

        for i in range(len(freqWord)):
            if freqWord[i] == 0:
                freqWord[i] = 1


        score = degreeWord/freqWord
        dic = {}

        for i in range(len(vocab)):        
            try:
                dic.update({vocab[i]: score[i]})
            except:
                print('error1')

        candidate_scores = []
        for element in clean_text:
            scr = 0
            for word in element:
                try:
                    scr += dic[word]
                except:
                    print()
            candidate_scores.append((scr, ' '.join(element)))

        candidate_scores.sort(reverse=True)
        l = (40*len(candidate_scores))//100
        candidate_scores = candidate_scores[0:l]
        key_phrase = []
        to_inlcude = ''
        count = -1000
        for i in candidate_scores:
            count = -1000
            for j in included_sentences:
                common = len( set( i[1].split(' ') ).intersection(set( j.split(' '))))
                if  common > count:
                    count  = common
                    to_inlcude = j
            key_phrase.append(to_inlcude)

        code_summary = ' '.join(key_phrase)
        gold_summary = pair[1]


        summaryBigram = utility.findGrams(code_summary.lower(), 1)
        humanBigram = utility.findGrams(gold_summary.lower(), 1)
        count = 0
        for element in humanBigram:
            if element in summaryBigram:
                count+=1

        result = count/len(humanBigram)
        combined.append(result)
    except:
        print ('Errored occured')


In [ ]:
sum(combined)/len(combined)

In [ ]:
combined.sort(reverse = True)
combined